In [1]:
import os
import sys
import pandas as pd
from tabulate import tabulate

(x_train,       # 00. wzorzec (wektor dł 256) np. array([10, 10, 52, 48, ...
    y_train,        # 01. etykieta, np. 'fsbndm'
    x_test,         # 02. jak x_train ale dla testow
    y_test,         # 03. jak y_train ale dla testow
    X_train_time,   # 04. DataFrame ze wszystkimi danymi dla kazdego wzorca
    X_test_time,    # 05. j.w. dla testowego
    Y_train_time,   # 06. chyba to samo co X_train_time
    Y_test_time,    # 07. chyba to samo co X_test_time
    y_train_onehot, # 08. one hot dla train
    y_test_onehot,  # 09. one hot dla test
    y_test_time,    # 10. czasy dla kazdego z testowanych algorytmow (w chwili pisania 27)
    y_train_time,   # 11. czasy dla kazdego z testowanych algorytmow (w chwili pisania 27)
    X_columns,      # 12. nazwy kolumn z kolejnymi symbolami wzorca np. pattern_0 (pierwszy znak), itd.
    alg_name_to_id, # 13. mapowanie nazwy algorytmu na jego ID
    algorithms,     # 14. Nazwy wszystkich algorytmow ktore zostaly przetestowane (czy moze wygraly przynajmniej raz?)
    algorithm_to_onehot
) = pd.read_pickle('dataset-full.pickle')

In [2]:
# Liczby wygranych dla kazdej dl wzorca oraz zbioru danych:
df = pd.concat([Y_train_time, Y_test_time])

In [3]:
# 3 najlepsze algorytmy w kategorii dataset, m
df = Y_test_time

tmp = df [['dataset', 'm', 'algorithm', 'pattern']].groupby(
    ['dataset', 'm', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()

n_best = 6 # czyli 3

records = []
for idx, gr in tmp.sort_values(by=['dataset', 'm', '#'], ascending=False).groupby(['dataset', 'm']):
    dataset, m = idx
    qwe = []
    coln = 0
    for idx2, el in gr.iterrows():
        _, _, alg, wins = el.values
        qwe += [[dataset, m, coln, f'{wins/5}%']]
        coln += 1
        qwe += [[dataset, m, coln, f'({alg})']]
        coln += 1
    qwe = qwe[:n_best]
    if len(qwe) < n_best:
        for i in range(len(qwe), n_best):
            qwe += [[dataset, m, coln, '']]
            coln += 1
            qwe += [[dataset, m, coln, '']]
            coln += 1
    records.extend(qwe)
records

df = pd.DataFrame(records)
df = df.rename(columns={0:'dataset', 1:'m', 2:'n-th best', 3:'result'})
df = df.pivot(index=['dataset', 'n-th best'], columns='m').reset_index()
df

dataset n-th best    result                      \
m                                                 4         6         8   
0   data/dblp.xml/dblp.xml.50MB         0     57.8%     64.8%     61.2%   
1   data/dblp.xml/dblp.xml.50MB         1    (ebom)    (ebom)    (ebom)   
2   data/dblp.xml/dblp.xml.50MB         2     41.2%     30.0%     31.4%   
3   data/dblp.xml/dblp.xml.50MB         3  (fsbndm)  (fsbndm)  (fsbndm)   
4   data/dblp.xml/dblp.xml.50MB         4      1.0%      5.2%      7.4%   
5   data/dblp.xml/dblp.xml.50MB         5   (sebom)   (sebom)   (sebom)   
6             data/dna/dna.50MB         0     37.6%     97.8%     97.6%   
7             data/dna/dna.50MB         1      (so)      (so)      (so)   
8             data/dna/dna.50MB         2     35.4%      2.2%      2.2%   
9             data/dna/dna.50MB         3    (ebom)    (ebom)    (ebom)   
10            data/dna/dna.50MB         4     12.4%                0.2%   
11            data/dna/dna.50MB         5  (fsbndm)             (askip)   
12            data/dna/dna.50MB         6       NaN                 NaN   
13            data/dna/dna.50MB         7       NaN                 NaN   
14            data/dna/dna.50MB         8       NaN       NaN       NaN   
15            data/dna/dna.50MB         9       NaN       NaN       NaN   
16    data/english/english.50MB         0     71.0%     84.8%     84.6%   
17    data/english/english.50MB         1    (ebom)    (ebom)    (ebom)   
18    data/english/english.50MB         2     29.0%     15.2%     15.4%   
19    data/english/english.50MB         3  (fsbndm)  (fsbndm)  (fsbndm)   
20    data/english/english.50MB         4                                 
21    data/english/english.50MB         5                                 
22    data/english/english.50MB         6                                 
23    data/english/english.50MB         7                                 
24    data/pitches/pitches.50MB         0     50.0%     62.2%     57.8%   
25    data/pitches/pitches.50MB         1    (ebom)    (ebom)    (ebom)   
26    data/pitches/pitches.50MB         2     47.6%     23.0%     21.6%   
27    data/pitches/pitches.50MB         3  (fsbndm)  (fsbndm)   (sebom)   
28    data/pitches/pitches.50MB         4      2.4%     14.8%     20.6%   
29    data/pitches/pitches.50MB         5   (sebom)   (sebom)  (fsbndm)   
30  data/proteins/proteins.50MB         0     87.0%     73.4%     50.4%   
31  data/proteins/proteins.50MB         1    (ebom)    (ebom)   (sebom)   
32  data/proteins/proteins.50MB         2      7.6%     22.2%     45.4%   
33  data/proteins/proteins.50MB         3  (fsbndm)   (sebom)    (ebom)   
34  data/proteins/proteins.50MB         4      5.4%      4.4%      4.2%   
35  data/proteins/proteins.50MB         5   (sebom)  (fsbndm)  (fsbndm)   
36  data/proteins/proteins.50MB         6       NaN       NaN       NaN   
37  data/proteins/proteins.50MB         7       NaN       NaN       NaN   
38    data/sources/sources.50MB         0     54.0%     63.4%     63.4%   
39    data/sources/sources.50MB         1  (fsbndm)    (ebom)    (ebom)   
40    data/sources/sources.50MB         2     46.0%     36.6%     36.6%   
41    data/sources/sources.50MB         3    (ebom)  (fsbndm)  (fsbndm)   
42    data/sources/sources.50MB         4                                 
43    data/sources/sources.50MB         5                                 
44    data/sources/sources.50MB         6                                 
45    data/sources/sources.50MB         7                                 

                                                                          \
m         10        12        14        16        24        32        64   
0      52.8%     43.4%     38.4%     46.0%     61.0%     56.0%     38.2%   
1     (ebom)    (ebom)    (ebom)  (fsbndm)  (fsbndm)  (fsbndm)      (br)   
2      32.8%     37.2%     38.0%     31.4%     18.2%     16.4%     26.0%   
3   (fsbndm)  (fsbndm)  (fsbndm)    (ebom) 

In [4]:
# 3 najlepsze algorytmy w kategorii dataset
df = Y_test_time

tmp = df [['dataset', 'algorithm', 'pattern']].groupby(
    ['dataset', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()

records = []
for idx, gr in tmp.sort_values(by=['dataset', '#'], ascending=False).groupby(['dataset']):
    dataset = idx
    qwe = []
    coln = 0
    for idx2, el in gr.iterrows():
        _, alg, wins = el.values
        qwe += [[dataset, coln, f'{round(wins/60, 2)}% ({alg})']]
        coln += 1
    qwe = qwe[:3]
    if len(qwe) < 3:
        for i in range(len(qwe), 3):
            qwe += [[dataset, coln, '']]
            coln += 1
    records.extend(qwe)
records


df = pd.DataFrame(records)
df = df.rename(columns={0:'dataset', 1:'n-th best', 2:'result'})
df = df.pivot(index=['dataset'], columns='n-th best').reset_index()
print(df.to_markdown(index=False))

| ('dataset', '')                  | ('result', 0)   | ('result', 1)   | ('result', 2)   |
|:---------------------------------|:----------------|:----------------|:----------------|
| ('data/dblp.xml/dblp.xml.50MB',) | 37.15% (fsbndm) | 34.15% (ebom)   | 16.47% (sebom)  |
| ('data/dna/dna.50MB',)           | 53.92% (askip)  | 31.47% (so)     | 5.15% (ebom)    |
| ('data/english/english.50MB',)   | 82.75% (ebom)   | 16.37% (fsbndm) | 0.63% (sebom)   |
| ('data/pitches/pitches.50MB',)   | 49.32% (ebom)   | 29.82% (sebom)  | 17.62% (fsbndm) |
| ('data/proteins/proteins.50MB',) | 67.02% (sebom)  | 31.03% (ebom)   | 1.93% (fsbndm)  |
| ('data/sources/sources.50MB',)   | 68.57% (ebom)   | 28.93% (fsbndm) | 2.02% (sebom)   |


In [5]:
# 3 najlepsze algorytmy w kategorii m
df = Y_test_time

tmp = df [['m', 'algorithm', 'pattern']].groupby(
    ['m', 'algorithm']
).count().rename(columns={'pattern': '#'}).reset_index()


records = []
for idx, gr in tmp.sort_values(by=['m', '#'], ascending=False).groupby(['m']):
    dataset = idx
    qwe = []
    coln = 0
    for idx2, el in gr.iterrows():
        _, alg, wins = el.values
        qwe += [[dataset, coln, f'{round(wins/60, 2)}% ({alg})']]
        coln += 1
    qwe = qwe[:3]
    if len(qwe) < 3:
        for i in range(len(qwe), 3):
            qwe += [[dataset, coln, '']]
            coln += 1
    records.extend(qwe)
records


df = pd.DataFrame(records)
df = df.rename(columns={0:'m', 1:'n-th best', 2:'result'})
df = df.pivot(index=['m'], columns='n-th best').reset_index()
print(df.to_markdown(index=False))

| ('m', '')   | ('result', 0)   | ('result', 1)   | ('result', 2)   |
|:------------|:----------------|:----------------|:----------------|
| (4,)        | 28.93% (ebom)   | 15.98% (fsbndm) | 3.13% (so)      |
| (6,)        | 29.23% (ebom)   | 9.1% (fsbndm)   | 8.15% (so)      |
| (8,)        | 26.22% (ebom)   | 9.02% (fsbndm)  | 8.13% (so)      |
| (10,)       | 25.85% (ebom)   | 8.1% (sebom)    | 8.02% (fsbndm)  |
| (12,)       | 23.53% (ebom)   | 10.25% (sebom)  | 8.4% (fsbndm)   |
| (14,)       | 22.23% (ebom)   | 11.17% (sebom)  | 9.1% (fsbndm)   |
| (16,)       | 20.62% (ebom)   | 11.62% (sebom)  | 9.58% (fsbndm)  |
| (24,)       | 16.5% (ebom)    | 13.13% (fsbndm) | 11.27% (sebom)  |
| (32,)       | 13.65% (ebom)   | 13.23% (fsbndm) | 11.33% (sebom)  |
| (64,)       | 20.07% (ebom)   | 13.53% (sebom)  | 8.62% (askip)   |
| (128,)      | 22.22% (ebom)   | 13.87% (sebom)  | 9.15% (askip)   |
| (256,)      | 21.92% (ebom)   | 13.95% (sebom)  | 6.92% (askip)   |
